<a href="https://colab.research.google.com/github/vibhamarpalle/Sanskrit_to_English_Translation/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-text tensorflow==2.17

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 68.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_text as text

tokenizer_sn = text.BertTokenizer("Sanskrit_unique_characters.txt")
tokenizer_en = text.BertTokenizer("English_unique_characters.txt")

In [ ]:
# import tensorflow as tf
# from transformers import BertTokenizer

# # Load the character-level tokenizers using the provided vocabulary files
# tokenizer_sn = BertTokenizer("/kaggle/input/char-vocab/sanskrit_vocab.txt", do_lower_case=False)
# tokenizer_en = BertTokenizer("/kaggle/input/char-vocab/english_vocab.txt", do_lower_case=False)

# # Example Sanskrit and English sentences
# sanskrit_sentence = 'ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्।'
# english_sentence = 'Hello, how are you?'

# sanskrit_tokens = list(sanskrit_sentence)
# english_tokens = list(english_sentence)

# # Convert tokens to IDs
# sanskrit_token_ids = tokenizer_sn.convert_tokens_to_ids(sanskrit_tokens)
# english_token_ids = tokenizer_en.convert_tokens_to_ids(english_tokens)

# # Display the results
# print("Sanskrit Tokens:", sanskrit_tokens)
# print("Sanskrit Token IDs:", sanskrit_token_ids)

# print("English Tokens:", english_tokens)
# print("English Token IDs:", english_token_ids)


In [ ]:
import pandas as pd
# df1=pd.read_csv('/kaggle/input/word-level/dict.csv')
df1=pd.read_csv('/content/dict.csv')
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df2=pd.read_csv('/content/correct_dict.csv')
df=pd.concat([df1,df2],ignore_index=True)
df=df.dropna()
df

,English,Sanskrit
0,I,अहम्
1,me,माम्
2,you,त्वम्
3,go,गच्छ
4,went,अगच्छत्
...,...,...
110348,"offered , came down",युयुजुह्
110349,you want to pierce,युयुन्क्ससि
110350,of those who are belligerent,युयुत्सतम्
110351,"the son of Dhrtarastra by his vaisya wife , S...",युयुत्सुह्


In [ ]:
df.sample(1)

,English,Sanskrit
31020,very difficult to pierce,दुर्भेद्यह्


In [ ]:
df['Sanskrit'].iloc[0]

'अहम्'

In [ ]:
def tokenize_char(s1):
    return ' '.join(list(s1))
print(tokenize_char('ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्। नारदं परिपप्रच्छ वाल्मीकिर्मुनिपुङ्गवम्॥'))

ॐ   त प ः   स ् व ा ध ् य ा य न ि र त ं   त प स ् व ी   व ा ग ् व ि द ा ं   व र म ् ।   न ा र द ं   प र ि प प ् र च ् छ   व ा ल ् म ी क ि र ् म ु न ि प ु ङ ् ग व म ् ॥


In [ ]:
df['Sanskrit']=df['Sanskrit'].apply(tokenize_char)
df['English']=df['English'].apply(tokenize_char)

In [ ]:
df['Sanskrit'].iloc[0]

'अ ह म ्'

In [ ]:
sanskrit_sentences = df['Sanskrit'].tolist()
english_sentences = df['English'].tolist()

In [ ]:
df['Sanskrit'].iloc[0]

'अ ह म ्'

In [ ]:
sanskrit_sentences[0]

'अ ह म ्'

In [ ]:
sanskrit_sentences[:10]

['अ ह म ्',
 'म ा म ्',
 'त ् व म ्',
 'ग च ् छ',
 'अ ग च ् छ त ्',
 'स ः',
 'स ा',
 'त त ्',
 'व य म ्',
 'त े']

In [ ]:
english_sentences[:10]

['I',
 'm e',
 'y o u',
 'g o',
 'w e n t',
 'h e',
 's h e',
 'i t',
 'w e',
 't h e y']

In [ ]:
tokenized_sanskrit = tokenizer_sn.tokenize(sanskrit_sentences)
tokenized_english = tokenizer_en.tokenize(english_sentences)

In [ ]:
tokenized_sanskrit[1]

<tf.RaggedTensor [[46],
 [56],
 [46],
 [67]]>

In [ ]:
type(tf.shape(tokenized_sanskrit))

tensorflow.python.ops.ragged.dynamic_ragged_shape.DynamicRaggedShape

In [ ]:
column_name = 'Sanskrit'

# Find the word with the maximum length in the specified column
df[column_name].dropna().loc[df[column_name].apply(len).idxmax()]

'स ् र द ् ध ,   म ै त ् र ि ,   द य ,   स न ् त ि ह ् ,   त ु स ् त ि ह ् ,   प ु स ् त ि ह ् ,   क ् र ि य ,   उ न ् न त ि ह ् ,   ब ्'

In [ ]:
sanskrit_padded = tokenized_sanskrit.to_tensor(default_value=0)  # No specific shape; default to max length
english_padded = tokenized_english.to_tensor(default_value=0)  # No specific shape; default to max length

max_len = 50
sanskrit_padded = tf.keras.preprocessing.sequence.pad_sequences(sanskrit_padded.numpy(), maxlen=max_len, padding='post', value=0)
english_padded = tf.keras.preprocessing.sequence.pad_sequences(english_padded.numpy(), maxlen=max_len, padding='post', value=0)

sanskrit_padded_tensor = tf.convert_to_tensor(sanskrit_padded)
english_padded_tensor = tf.convert_to_tensor(english_padded)

print(sanskrit_padded_tensor.shape)
print(english_padded_tensor.shape)

(110353, 50, 1)
(110353, 50, 1)


In [ ]:
import numpy as np

def positional_encoding(max_len, embedding_dim):
    pos_enc = np.zeros((max_len, embedding_dim))
    for pos in range(max_len):
        for i in range(0, embedding_dim, 2):
            pos_enc[pos, i] = np.sin(pos / (10000 ** (i / embedding_dim)))
            pos_enc[pos, i + 1] = np.cos(pos / (10000 ** ((i + 1) / embedding_dim)))
    return pos_enc


In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense, MultiHeadAttention, LayerNormalization, Dropout

def transformer_encoder(vocab_size, embedding_dim, num_heads, ff_dim, max_len, dropout_rate=0.1):
    input_seq = Input(shape=(max_len,))
    embedding = Embedding(vocab_size, embedding_dim)(input_seq)
#     pos_encoding = positional_encoding(max_len, embedding_dim)
#     embedding += pos_encoding

    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(embedding, embedding)
    attn_output = Dropout(dropout_rate)(attn_output)
    attn_output = LayerNormalization(epsilon=1e-6)(attn_output + embedding)

    ff_output = Dense(ff_dim, activation='sigmoid')(attn_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Dense(embedding_dim)(ff_output)
    ff_output = LayerNormalization(epsilon=1e-6)(ff_output + attn_output)

    return tf.keras.Model(inputs=input_seq, outputs=ff_output)


def transformer_decoder(vocab_size, embedding_dim, num_heads, ff_dim, max_len, dropout_rate=0.1):
    input_seq = Input(shape=(max_len,))
    encoder_output = Input(shape=(max_len, embedding_dim))

    embedding = Embedding(vocab_size, embedding_dim)(input_seq)
#     pos_encoding = positional_encoding(max_len, embedding_dim)
#     embedding += pos_encoding

    attn1 = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(embedding, embedding)
    attn1 = Dropout(dropout_rate)(attn1)
    attn1 = LayerNormalization(epsilon=1e-6)(attn1 + embedding)

    attn2 = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(attn1, encoder_output)
    attn2 = Dropout(dropout_rate)(attn2)
    attn2 = LayerNormalization(epsilon=1e-6)(attn2 + attn1)

    ff_output = Dense(ff_dim, activation='relu')(attn2)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Dense(embedding_dim)(ff_output)
    ff_output = LayerNormalization(epsilon=1e-6)(ff_output + attn2)

    return tf.keras.Model(inputs=[input_seq, encoder_output], outputs=ff_output)

In [ ]:
def build_transformer(vocab_size_sn, vocab_size_en, embedding_dim, num_heads, ff_dim, max_len):
    encoder_inputs = Input(shape=(max_len,))
    decoder_inputs = Input(shape=(max_len,))

    encoder_model = transformer_encoder(vocab_size_sn, embedding_dim, num_heads, ff_dim, max_len)
    encoder_output = encoder_model(encoder_inputs)

    decoder_model = transformer_decoder(vocab_size_en, embedding_dim, num_heads, ff_dim, max_len)
    decoder_output = decoder_model([decoder_inputs, encoder_output])

    final_output = Dense(vocab_size_en, activation='softmax')(decoder_output)

    return tf.keras.Model([encoder_inputs, decoder_inputs], final_output)


embedding_dim = 256
num_heads = 8
ff_dim = 512
max_len = 50

def load_vocab_size(vocab_file):
    with open(vocab_file, 'r', encoding='utf-8') as f:
        vocab = f.readlines()
    return len(vocab)


vocab_size_sn = load_vocab_size("English_unique_characters.txt")  # Sanskrit vocabulary size
vocab_size_en = load_vocab_size("Sanskrit_unique_characters.txt")  # English vocabulary size

In [ ]:
print("Max index in Sanskrit padded:", np.max(sanskrit_padded))
print("Max index in English padded:", np.max(english_padded))
print("Sanskrit vocab size:", vocab_size_sn)
print("English vocab size:", vocab_size_en)

vocab_size_sn = max(np.max(sanskrit_padded) + 1, vocab_size_sn)
vocab_size_en = max(np.max(english_padded) + 1, vocab_size_en)

print("Max index in Sanskrit padded:", np.max(sanskrit_padded))
print("Max index in English padded:", np.max(english_padded))
print("Sanskrit vocab size:", vocab_size_sn)
print("English vocab size:", vocab_size_en)

Max index in Sanskrit padded: 67
Max index in English padded: 72
Sanskrit vocab size: 74
English vocab size: 69
Max index in Sanskrit padded: 67
Max index in English padded: 72
Sanskrit vocab size: 74
English vocab size: 73


In [ ]:
model = build_transformer(vocab_size_sn, vocab_size_en, embedding_dim, num_heads, ff_dim, max_len)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 50)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 50)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, 50, 256)        │      2,386,432 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_1 (Functional) │ (None, 50, 256)        │      4,490,240 │ input_layer_1[0][0],   │
│                           │                        │                │ functional[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 50, 73)         │         18,761 │ functional_1[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 6,895,433 (26.30 MB)

 Trainable params: 6,895,433 (26.30 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
batch_size = 64
epochs = 3

train_size = int(0.8 * len(sanskrit_padded))
x_train_sn = sanskrit_padded[:train_size]
x_val_sn = sanskrit_padded[train_size:]
x_train_en = english_padded[:train_size]
x_val_en = english_padded[train_size:]

history = model.fit([x_train_sn, x_train_en], x_train_en,
                    validation_data=([x_val_sn, x_val_en], x_val_en),
                    batch_size=batch_size,
                    epochs=epochs)

Epoch 1/3
1380/1380 ━━━━━━━━━━━━━━━━━━━━ 112s 68ms/step - accuracy: 0.9943 - loss: 0.0310 - val_accuracy: 1.0000 - val_loss: 8.3446e-07
Epoch 2/3
1380/1380 ━━━━━━━━━━━━━━━━━━━━ 124s 64ms/step - accuracy: 1.0000 - loss: 8.6202e-06 - val_accuracy: 1.0000 - val_loss: 1.9312e-05
Epoch 3/3
1380/1380 ━━━━━━━━━━━━━━━━━━━━ 139s 62ms/step - accuracy: 1.0000 - loss: 4.3911e-04 - val_accuracy: 1.0000 - val_loss: 5.9605e-07


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# m1=load_model('/kaggle/working/transformer_gita.h5')

In [ ]:
model.save('transformer_gita.h5')

In [ ]:
# model.save('p1.h5')

In [ ]:
import numpy as np
import tensorflow as tf

def predict_translation(input_sentence, tokenizer_sn, tokenizer_en, model, max_len=50):
    tokenized_input = tokenizer_sn.tokenize([input_sentence])
    print(tokenized_input)
    input_padded = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_input.to_tensor(default_value=0).numpy(),
        maxlen=max_len,
        padding='post',
        value=0
    )
    input_padded = tf.convert_to_tensor(input_padded, dtype=tf.int32)
    print(input_padded)

    start_token = [1]
    decoder_input = tf.keras.preprocessing.sequence.pad_sequences(
        [start_token],
        maxlen=max_len,
        padding='post',
        value=0
    )
    decoder_input = tf.convert_to_tensor(decoder_input, dtype=tf.int32)

    predicted_sentence = []

    # prediction = model.predict([input_padded, decoder_input])
    # print(prediction)

    # for i in range(max_len):
    #   next_token = np.argmax(prediction[0][i])
    #   print(next_token)

    #   predicted_sentence.append(next_token)

    # translated_sentence = tokenizer_en.detokenize(predicted_sentence)
    # return translated_sentence

    for _ in range(max_len):
        prediction = model.predict([input_padded, decoder_input])
        print(prediction)
        print(len(prediction[0]))
        next_token = np.argmax(prediction[:, -1, :], axis=-1)[0]

        decoder_input[0][-1] = next_token

        if next_token == 2:
            break

        predicted_sentence.append(next_token)

    print(predicted_sentence)

    if predicted_sentence:
        predicted_sentence = np.array(predicted_sentence).reshape(1, -1)  # Reshape to 2D
    else:
        predicted_sentence = np.array([]).reshape(1, 0)  # Handle empty case

    translated_sentence = tokenizer_en.detokenize(predicted_sentence)

    return translated_sentence


In [ ]:
model.save('t1_gita.keras')

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [ ]:
from tensorflow.keras.models import load_model
# input_sentence = 'आत्मवान् को जितक्रोधो द्युतिमान् कोऽनसूयकः। कस्य बिभ्यति देवाश्च जातरोषस्य संयुगे॥'
# input_sentence=' '.join(list(input_sentence))
input_sentence = "अगच्छत्"
input_sentence = tokenize_char(input_sentence)
print(input_sentence)
m1=load_model('t1_gita.keras')
translated = predict_translation(input_sentence, tokenizer_sn, tokenizer_en, m1)
print(translated)


अ ग च ् छ त ्
<tf.RaggedTensor [[[10],
  [24],
  [27],
  [67],
  [28],
  [37],
  [67]]]>
tf.Tensor(
[[[10]
  [24]
  [27]
  [67]
  [28]
  [37]
  [67]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]]], shape=(1, 50, 1), dtype=int32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[[9.9999940e-01 4.0803934e-09 6.2104153e-09 ... 6.3016936e-10
   9.9444653e-10 2.5292199e-08]
  [9.9999940e-01 4.0807979e-09 6.2121095e-09 ... 6.3037853e-10
   9.9461539e-10 2.5289451e-08]
  [9.9999940e-01 4.0807979e-09 6.2121095e-09 ... 6.3037853e-10
   9.9461539e-10 2.5289451e-08]
  ...
  [9.9999940e-01 4.0807979e-09 6.2121095e-09 ... 6.3037853e-10
   9.9461539e-10 2.5289451e-08]
  [9.9999940e-01 4.0807979e-09 6.2121095e-09 ... 6.3037853e-10
   9.9461539e-10 2.5289451e-08]

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment